# Fondamenti di elaborazione immagini

## Effettuiamo l'import delle librerie utilizzate nell'esercitazione.

In [ ]:
import cv2
import numpy as np
import matplotlib as mapli
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

Di seguito i riferimenti alle pagine di documentazione, sempre utiliti:

* Rif: [numpy](https://numpy.org/doc/stable/)
* Rif: [opencv](https://docs.opencv.org/)
* Rif: [matplotlib](https://matplotlib.org/stable/index.html)

Aggiungiamo alcune funzioni di utilita' per la scrittura del codice.

In [ ]:
def rgb(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def gray(image : np.array) -> np.array:
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def grid(images : list[np.array], rows : int, cols : int, size : int, colors : list[str] = None) -> None:
    fig = plt.figure(figsize=(size,size))
    grid = ImageGrid(fig, 111, nrows_ncols=(rows, cols), axes_pad=0.1)

    if colors is not None:
        counter = 0
        for ax, im in zip(grid, images):
            ax.imshow(im, cmap=colors[counter])
            counter = (counter + 1) % len(colors)
        plt.show()
    else:
        for ax, im in zip(grid, images):
            ax.imshow(im)
        plt.show()

## _La convoluzione e' una fra le piu' note tecniche di trasformazione delle immagini, utilizzata sia come tecnica di preprocessing che come tecnica applicabile all'ambito del deep learning._

Per iniziare questa operazione e' necessario definire una matrice di convoluzione o kernel. Grazie all'interscambiabilita' presente fra _numpy_ e _opencv_, e' possibile farlo definendo direttamente la matrice di numeri. Facciamo riferimento ad alcune delle matrici definite dal seguente link wikipedia:

* Rif: [Convolution kernel](https://en.wikipedia.org/wiki/Kernel_(image_processing))

In [ ]:
kernel_sample = np.array([[0, 0, 0],
                          [0, 1, 0],
                          [0, 0, 0]])

print(kernel_sample)

In [ ]:
image_sample = np.array([[1, 2, 3],
                         [4, 5, 6],
                         [7, 8, 9]]).astype(np.uint8)

print(image_sample)

Il kernel di esempio definito e':
* Quadrato di dimensione 3x3.
* Simmetrico con il centro in (1,1)
Per come sono stati definiti i suoi valori, ogni volta che si aggancera' ad un pixel, ne manterra' il valore senza tenere conto del 'vicinato'.

In [ ]:
convolution = cv2.filter2D(src=image_sample, ddepth=-1, kernel=kernel_sample)
print(convolution)

Per applicare il filtro di convoluzione, il kernel, all'operazione di convoluzione, abbiamo utilizzato il metodo _filter2D_ di _opencv_. Gli argomenti passati sono semplicemente l'immagine, il kernel e la dimensione attesa per l'output. Quest'ultimo parametro, indicato da **ddepth**, se posto a -1 indica che l'output avra' stessa dimensione dell'input.

* Rif: [filter2D](https://docs.opencv.org/4.x/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04)

## _Solamente cambiando la matrice di valori, i risultati ottenuti possono variare._ 

Proviamo ad esempio diverse tipologie di kernel. Creiamo una funzione di utilita' per semplificarci le prove.

In [ ]:
def rgb_convolution(image : str, kernel : np.ndarray, delta : int = 0, threshold : int = 30) -> None:

    # legge l'immagine a colori
    image = cv2.imread(image, cv2.IMREAD_COLOR)

    # applichiamo il filtro di convoluzione.
    convolution = cv2.filter2D(src=image, ddepth=-1, kernel=kernel, delta=delta)

    # applichiamo una sogliatura per mostrare al meglio il risultato
    _, thresholded = cv2.threshold(convolution, threshold, 255, cv2.THRESH_BINARY)

    grid([rgb(image), rgb(convolution), thresholded], 1, 3, 25)

Carichiamo un'immagine di esempio.

In [ ]:
image = './imgs/kitten.png'
delta = 0
threshold = 20

In [ ]:
# Proviamo un kernel di ricerca gradienti.
kernel = np.array([[-1, -1, -1],
                   [-1,  8, -1],
                   [-1, -1, -1]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di ricerca gradienti.
kernel = np.array([[ 0, -1,  0],
                   [-1,  4, -1],
                   [ 0, -1,  0]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di ricerca linee orizzontali.
kernel = np.array([[-1, -1, -1],
                   [ 2,  2,  2],
                   [-1, -1, -1]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di ricerca linee verticali.
kernel = np.array([[-1, 2, -1],
                   [-1, 2, -1],
                   [-1, 2, -1]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di ricerca linee a -45°.
kernel = np.array([[2, -1, -1],
                   [-1, 2, -1],
                   [-1, -1, 2]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di ricerca linee a 45°.
kernel = np.array([[-1, -1,  2],
                   [-1,  2, -1],
                   [ 2, -1, -1]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di differenza.
kernel = np.array([[ 0,  0,  0],
                   [ 0,  1, -1],
                   [ 0,  0,  0]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di differenza.
kernel = np.array([[ 0,  0,  0],
                   [ 1,  0, -1],
                   [ 0,  0,  0]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di differenza. (Roberts)
kernel = np.array([[ 0,  0, -1],
                   [ 0,  1,  0],
                   [ 0,  0,  0]])

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di differenza. (Prewitt)
kernel = (np.array([[ 1, 0, -1],
                    [ 1, 0, -1],
                    [ 1, 0, -1]]).astype(np.float32)) / 3

rgb_convolution(image, kernel, delta, threshold)

In [ ]:
# Proviamo un kernel di differenza. (Sobel)
kernel = (np.array([[ 1, 0, -1],
                    [ 2, 0, -2],
                    [ 1, 0, -1]]).astype(np.float32)) / 4

rgb_convolution(image, kernel, delta, threshold)

Per applicare una semplice sfocatura, si puo', ad esempio, assegnare ad ogni pixel un valore dato dall'equo contributo di tutti i suoi vicini (lui stesso compreso). Con un kernel quadrato 3x3, centrato sul pixel, si andra' a considerare lui e tutti i suoi vicini. 

In [ ]:
# Proviamo un kernel di sfocatura.
kernel = (np.array([[ 1, 1, 1],
                    [ 1, 1, 1],
                    [ 1, 1, 1]]).astype(np.float32)) / 9

rgb_convolution(image, kernel, 0, 255)

Per esagerare con la sfocatura, possiamo chiedere il contributo di un vicinato piu' grande: ad esempio un 7x7 attorno al pixel in esame.

In [ ]:
# Proviamo un kernel di sfocatura.
kernel = np.array([[1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 1]]).astype(np.float32) / 49

rgb_convolution(image, kernel, 0, 255)

Aumentiamo il kernel.

In [ ]:
# Proviamo un kernel di sfocatura.
kernel = np.ones((9, 9), np.float32) / 81

rgb_convolution(image, kernel, 0, 255)

In [ ]:
# Proviamo un kernel di smoothing.
kernel = np.array([[ 0, -1,  0],
                   [-1,  5, -1],
                   [ 0, -1,  0]])

rgb_convolution(image, kernel, 0, 255)